In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['TRANSFORMERS_CACHE'] = '/mnt/iMVR/junde/.cache/huggingface/hub'

from langchain.llms import HuggingFacePipeline
import networkx as nx
from transformers import pipeline
import re
from creat import creat_world
from prompt import *
from server import logger
from datetime import datetime


logger.configure(dir = './log/log-' + str(datetime.now()))
logger.log("creating data loader...")

Logging to ./log/log-2023-12-12 04:03:02.065507
creating data loader...


In [4]:
pipeline = pipeline(model="meta-llama/Llama-2-7b-chat-hf", device_map="auto")
model = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.51s/it]


In [5]:
locations, agents = creat_world(model)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, sup

In [6]:
action_results = {}
back_know = []
for i in town_people.keys(): # generate bk knowledge for everyone
  back_know.append(i+': '+ agents[i].get_summary(force_refresh=False))
# bk = [x for x in back_know]
for i in town_people.keys(): # add to people mem
  agents[i].memory.add_memory(' You know the following about people: ' + '. '.join(back_know))
  action_results[i] = i + 'is' + town_people[i]["status"]

In [14]:
print(' You know the following about people: ' + '. '.join(back_know))

 You know the following about people: Postowner Lius: Name: Postowner Lius (age: 69)
Innate traits: mean, gossipy, arrogant, selfish
Postowner Lius owns a trading post.. Retireadven Daran: Name: Retireadven Daran (age: 55)
Innate traits: Stubborn, upright, inarticulate, calm
Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neverwinter region, his original home.. Postmanager Linene: Name: Postmanager Linene (age: 38)
Innate traits: competent, forbearing, self-discipline, impressionable 
Linene runs a trading post.. ToweMaster Harbin: Name: ToweMaster Harbin (age: 40)
Innate traits: pompous
Harbin is the townmaster of Phandalin. A pompous, old food. Phandalin has no functioning government, but the townsfolk elect someone to serve a

In [8]:
global_time = 0
for repeats in range(5):
  global_time += 1
  print("In global time", global_time)
  action_prompts = {}
  people_description = []
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
        people_description.append(action_results[i])

    # for name in people:
      # prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
    # for i in people: 
    #   if i not in action_results: # initialize action results as the status, and add agent mem with description
    #     action_results[i] = agents[i].get_summary(force_refresh=False)
    for i in people: # add observation to memory and react
      logger.log("Mind Tree of people: ", i)

      others = [x for x in people if x != i]
      observation = "You are {}.You are currently in {} with the following description: {}. \
      It is currently {}:00. The following people are in this area: {}. You can interact with them.". \
      format(i, location, town_areas[location], str(global_time), ', '.join(others))

      others_des = [x for x in people_description if i+': ' not in x]
      observation += ' You know the following about people: ' + '. '.join(others_des)

      logger.log("For people %s, The observation is: %s \n" % (i, observation))

      agents[i].memory.add_memory(observation)
      _, reaction = agents[i].generate_reaction(observation)
      action_results[i] = reaction

      logger.log("action result is:  %s \n" %(reaction))

In global time 1
Mind Tree of people:  Postowner Lius
For people Postowner Lius, The observation is: You are Postowner Lius.You are currently in Phandalin Town Square with the following description: Town square of the town of Phandalin..       It is currently 1:00. The following people are in this area: Retireadven Daran, Postmanager Linene, ToweMaster Harbin, Robber Terrill. You can interact with them. You know the following about people: Postowner Liusischatting with Post-Manager. Retireadven Daranisdriking the beer. Postmanager Lineneischatting with Postowner Lius. ToweMaster Harbiniscalculating the tax of the town. Robber Terrillischating with Redbrand ruffians members, secretly planing the robbery 

the entity i care most is: Postowner Lius 

the entities' action is: chatting with the Post-Manager Linene in the town square of Phandalin. 

agent_summary_description is: Name: Postowner Lius (age: 69)
Innate traits: mean, gossipy, arrogant, selfish
Postowner Lius owns a trading post.

/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


action result is:  Postowner Lius scowls at Robber Terrill, muttering under his breath. 

Mind Tree of people:  Retireadven Daran
For people Retireadven Daran, The observation is: You are Retireadven Daran.You are currently in Phandalin Town Square with the following description: Town square of the town of Phandalin..       It is currently 1:00. The following people are in this area: Postowner Lius, Postmanager Linene, ToweMaster Harbin, Robber Terrill. You can interact with them. You know the following about people: Postowner Liusischatting with Post-Manager. Retireadven Daranisdriking the beer. Postmanager Lineneischatting with Postowner Lius. ToweMaster Harbiniscalculating the tax of the town. Robber Terrillischating with Redbrand ruffians members, secretly planing the robbery 



/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


the entity i care most is: Retireadven Daran 



...

agent_summary_description is: Name: ToweMaster Harbin (age: 40)
Innate traits: pompous
Harbin is the townmaster of Phandalin. A pompous, old food. Phandalin has no functioning government, but the townsfolk elect someone to serve as townmaster each year. The townmaster serves as a judge in minor disputes and keeps any records that need to be kept. 

relevant_mem is: - Phandalin has no functioning government, but the townsfolk elect someone to serve as townmaster each year. 
- The townmaster serves as a judge in minor disputes and keeps any records that need to be kept. 
- Robber Terrill is a member of the Redbrand ruffians and wants to rob and kill adventurers.
- Postowner Lius, Retireadven Daran, Postmanager Linene, and ToweMaster Harbin are all present in the town square.
- Robber Terrill is secretly planning a robbery with the Redbrand ruffians.
- The year is 1207 DR, and the events of the campaign take place in the spring of this year.
- The town of Phandalin is located in the Never

/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


full action logic is: Please let me know if you need more information or context. 



/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


KeyboardInterrupt: 